In [1]:
import os
import re
import glob
# import seaborn as sns
import numpy as np
import pandas as pd
# import plotly.express as px

from datetime import date
# from onc_db_tools.comp_gen_db import CosmicClient, DepMapClient


In [2]:
import onc_db_tools

ModuleNotFoundError: No module named 'onc_db_tools'

In [4]:
# from onc_db_tools.cbio_portal import CbioPortalClient
# from onc_db_tools.utils import group_count
# from onc_db_tools.utils import process_depmap_mutations
# from onc_db_tools.ml_analysis import FeatureImportanceEval

# %matplotlib inline
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


# aws_access_df = pd.read_csv("/home/ec2-user/.ssh/aws_athena_access_keys.csv")

# AWS_ACCESS_KEY = aws_access_df['access_key'].values[0]
# AWS_SECRET_KEY = aws_access_df['secret_key'].values[0]

#generate time stamp
today = date.today()
date_string = today.strftime("%Y-%m-%d")

print(date_string)

2025-06-11


### Setting up CompGen MetaRep

In [2]:
from bbio.reporting.s3 import get_analysis_path
from bbio.reporting.airtable import upsert_analysis_metadata


analysis_id = "CG-1010"

S3_PATH = get_analysis_path(analysis_id = analysis_id, 
                                filesystem = "s3")

project_name_default = os.path.basename(os.path.abspath("../"))

upsert_analysis_metadata(result_path=S3_PATH, 
                         user='jin.ju@bridgebio.com',
                         task_id=analysis_id,
                         project_name = "")

In [3]:
%config Completer.use_jedi = False

### Setting Up Google BigQuery

In [3]:
### Open Targets Platform Through BigQuery

import os
from google.cloud.bigquery.client import Client

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/jonathancheng/Documents/code/onc-db-tools/bbio-bigquery-settings.json"
client = Client()


query = """
SELECT 
id as target_id,
approvedSymbol as target_symbol,
tractability.element.modality as modality,
tractability.element.id as id,
tractability.element.value as value,
FROM open-targets-prod.platform.target
JOIN
  UNNEST(tractability.list) as tractability
"""
tractability_df = client.query(query).to_dataframe(create_bqstorage_client=True)


/Users/jonathancheng/miniconda3/envs/conda_dev/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
tractability_df.head()

,target_id,target_symbol,modality,id,value
0,ENSG00000278152,KIR2DS2,SM,Approved Drug,False
1,ENSG00000278152,KIR2DS2,SM,Advanced Clinical,False
2,ENSG00000278152,KIR2DS2,SM,Phase 1 Clinical,False
3,ENSG00000278152,KIR2DS2,SM,Structure with Ligand,False
4,ENSG00000278152,KIR2DS2,SM,High-Quality Ligand,False


In [ ]:
### Open Targets Platform Through BigQuery

# this works

import os
from google.cloud.bigquery.client import Client

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/jonathancheng/Documents/code/onc-db-tools/bbio-bigquery-settings.json"
client = Client()


query = """
SELECT
  associations.targetId AS target_id,
  targets.approvedSymbol AS target_approved_symbol,
  associations.diseaseId AS disease_id,
  diseases.name AS disease_name,
  associations.score AS overall_association_score
FROM
  `open-targets-prod.platform.association_overall_direct` AS associations
JOIN
  `open-targets-prod.platform.disease` AS diseases
ON
  associations.diseaseId = diseases.id
JOIN
  `open-targets-prod.platform.target` AS targets
ON
  associations.targetId = targets.id
WHERE
  associations.diseaseId='EFO_0000676'
ORDER BY
  associations.score DESC
"""
tractability_df = client.query(query).to_dataframe(create_bqstorage_client=True)


/Users/jonathancheng/miniconda3/envs/conda_dev/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [2]:
tractability_df.head()

,target_id,target_approved_symbol,disease_id,disease_name,overall_association_score
0,ENSG00000113302,IL12B,EFO_0000676,psoriasis,0.773410
1,ENSG00000105397,TYK2,EFO_0000676,psoriasis,0.723711
2,ENSG00000056972,TRAF3IP2,EFO_0000676,psoriasis,0.694501
3,ENSG00000110944,IL23A,EFO_0000676,psoriasis,0.663955
4,ENSG00000177663,IL17RA,EFO_0000676,psoriasis,0.651265


In [5]:
tractability_df.head(10)

,target_id,target_symbol,modality,id,value
0,ENSG00000278304,KIR2DS1,SM,Approved Drug,False
1,ENSG00000278304,KIR2DS1,SM,Advanced Clinical,False
2,ENSG00000278304,KIR2DS1,SM,Phase 1 Clinical,False
3,ENSG00000278304,KIR2DS1,SM,Structure with Ligand,False
4,ENSG00000278304,KIR2DS1,SM,High-Quality Ligand,False
5,ENSG00000278304,KIR2DS1,SM,High-Quality Pocket,False
6,ENSG00000278304,KIR2DS1,SM,Med-Quality Pocket,False
7,ENSG00000278304,KIR2DS1,SM,Druggable Family,False
8,ENSG00000278304,KIR2DS1,AB,Approved Drug,False
9,ENSG00000278304,KIR2DS1,AB,Advanced Clinical,False


In [6]:
tractability_df['id_clean'] = [x.lower().replace(" ","_") for x in tractability_df['id']]

In [7]:
tractability_df['column_names'] = tractability_df['modality'] + "_" + tractability_df['id_clean']

In [8]:
tractability_df.to_csv(os.path.join("s3://bbio-data-platform/dev/data_driven_target_disease_prioritization/summary_tables/2023-02-03/", 
                                    f"OpenTargets_target_tractability_long_{date_string}.csv"), index = False)

In [9]:
tractability_df['column_names'].unique()

array(['SM_approved_drug', 'SM_advanced_clinical', 'SM_phase_1_clinical',
       'SM_structure_with_ligand', 'SM_high-quality_ligand',
       'SM_high-quality_pocket', 'SM_med-quality_pocket',
       'SM_druggable_family', 'AB_approved_drug', 'AB_advanced_clinical',
       'AB_phase_1_clinical', 'AB_uniprot_loc_high_conf',
       'AB_go_cc_high_conf', 'AB_uniprot_loc_med_conf',
       'AB_uniprot_sigp_or_tmhmm', 'AB_go_cc_med_conf',
       'AB_human_protein_atlas_loc', 'PR_approved_drug',
       'PR_advanced_clinical', 'PR_phase_1_clinical', 'PR_literature',
       'PR_uniprot_ubiquitination', 'PR_database_ubiquitination',
       'PR_half-life_data', 'PR_small_molecule_binder',
       'OC_approved_drug', 'OC_advanced_clinical', 'OC_phase_1_clinical'],
      dtype=object)

In [11]:
tractability_wide = tractability_df.pivot_table(index = ['target_id', 'target_symbol'], columns = 'column_names', values = 'value')

In [22]:
tractability_df.head()

,target_id,target_symbol,modality,id,value,id_clean,column_names
0,ENSG00000278304,KIR2DS1,SM,Approved Drug,False,approved_drug,SM_approved_drug
1,ENSG00000278304,KIR2DS1,SM,Advanced Clinical,False,advanced_clinical,SM_advanced_clinical
2,ENSG00000278304,KIR2DS1,SM,Phase 1 Clinical,False,phase_1_clinical,SM_phase_1_clinical
3,ENSG00000278304,KIR2DS1,SM,Structure with Ligand,False,structure_with_ligand,SM_structure_with_ligand
4,ENSG00000278304,KIR2DS1,SM,High-Quality Ligand,False,high-quality_ligand,SM_high-quality_ligand


In [17]:
tractability_wide.head()

,column_names,AB_advanced_clinical,AB_approved_drug,AB_go_cc_high_conf,AB_go_cc_med_conf,AB_human_protein_atlas_loc,AB_phase_1_clinical,AB_uniprot_loc_high_conf,AB_uniprot_loc_med_conf,AB_uniprot_sigp_or_tmhmm,OC_advanced_clinical,OC_approved_drug,OC_phase_1_clinical,PR_advanced_clinical,PR_approved_drug,PR_database_ubiquitination,PR_half-life_data,PR_literature,PR_phase_1_clinical,PR_small_molecule_binder,PR_uniprot_ubiquitination,SM_advanced_clinical,SM_approved_drug,SM_druggable_family,SM_high-quality_ligand,SM_high-quality_pocket,SM_med-quality_pocket,SM_phase_1_clinical,SM_structure_with_ligand
target_id,target_symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,TSPAN6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000005,TNMD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000419,DPM1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000457,SCYL3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000460,C1orf112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Integrate full Kenan et al. Table

In [61]:
ref_table = pd.read_csv("../resource_files/kenan_druggable_genome_list_20170529.csv").rename({"ensembl_gene_id" : 'target_id',
                                                                                              'hgnc_names' : 'target_symbol'}, axis = 1)

In [69]:
gene_mapping = pd.read_csv("s3://bbio-data-platform/dev/gene_id_mapping/human_gene_mapping_entrez_ensembl_symbol.csv")

In [70]:
gene_mapping.head()

,target_id,target_symbol,target_entrez
0,ENSG00000279061,ENSG00000279061,NaN
1,ENSG00000279002,ENSG00000279002,NaN
2,ENSG00000280177,ENSG00000280177,NaN
3,ENSG00000278875,ENSG00000278875,NaN
4,ENSG00000279326,ENSG00000279326,NaN


In [83]:
ref_table_id = ref_table.merge(gene_mapping[['target_id', 'target_symbol']], 
                              on = 'target_symbol',
                              suffixes = ("_kenan", ""))

In [84]:
ref_table_id.query("target_id_kenan != target_id").shape

(66, 13)

In [85]:
ot_sm_druggable = tractability_df.loc[(tractability_df['column_names'] == 'SM_druggable_family') & (tractability_df['value'])].drop_duplicates('target_id')

In [90]:
ref_table_id['SM_druggable'] = ref_table_id['small_mol_druggable'] == 'Y'
ref_table_id['AB_druggable'] = ref_table_id['bio_druggable'] == 'Y'
ref_table_id['is_adme_gene'] = ref_table_id['adme_gene'] == 'Y'


In [91]:
ref_table_id[['target_id', 'druggability_tier', 
              'SM_druggable', 'AB_druggable', 'is_adme_gene']]

,target_id,druggability_tier,SM_druggable,AB_druggable,is_adme_gene
0,ENSG00000000938,Tier 1,True,False,False
1,ENSG00000001626,Tier 1,True,False,True
2,ENSG00000001630,Tier 1,True,False,True
3,ENSG00000002549,Tier 1,True,False,False
4,ENSG00000004468,Tier 1,False,True,False
...,...,...,...,...,...
4415,ENSG00000276085,Tier 3B,False,True,False
4416,ENSG00000276085,Tier 3B,False,True,False
4417,ENSG00000258818,Tier 3B,False,True,False
4418,ENSG00000259803,Tier 3B,True,False,False


In [92]:
relevant_columns = ['SM_structure_with_ligand', 'SM_high-quality_ligand',
                    'SM_high-quality_pocket', 'SM_med-quality_pocket',
                    'AB_uniprot_loc_high_conf',
                    'AB_go_cc_high_conf', 'AB_uniprot_loc_med_conf',
                    'AB_uniprot_sigp_or_tmhmm', 'AB_go_cc_med_conf',
                    'AB_human_protein_atlas_loc', 'PR_literature',
                    'PR_uniprot_ubiquitination', 'PR_database_ubiquitination',
                    'PR_half-life_data', 'PR_small_molecule_binder']

tractability_save = ref_table_id[['target_id', 'druggability_tier', 
                                  'SM_druggable', 'AB_druggable', 
                                  'is_adme_gene']].merge(tractability_wide[relevant_columns].reset_index(),
                                                                     on = 'target_id', how = 'right')
                                   

In [95]:
date_string

'2023-03-10'

In [96]:
tractability_save.to_csv(os.path.join(f"s3://bbio-data-platform/dev/data_driven_target_disease_prioritization/summary_tables/opentargets_target_tractability_wide/{date_string}/", 
                                      f"opentargets_target_tractability_wide.csv"), index = False)

In [89]:
tractability_wide.head()

,column_names,AB_advanced_clinical,AB_approved_drug,AB_go_cc_high_conf,AB_go_cc_med_conf,AB_human_protein_atlas_loc,AB_phase_1_clinical,AB_uniprot_loc_high_conf,AB_uniprot_loc_med_conf,AB_uniprot_sigp_or_tmhmm,OC_advanced_clinical,OC_approved_drug,OC_phase_1_clinical,PR_advanced_clinical,PR_approved_drug,PR_database_ubiquitination,PR_half-life_data,PR_literature,PR_phase_1_clinical,PR_small_molecule_binder,PR_uniprot_ubiquitination,SM_advanced_clinical,SM_approved_drug,SM_druggable_family,SM_high-quality_ligand,SM_high-quality_pocket,SM_med-quality_pocket,SM_phase_1_clinical,SM_structure_with_ligand
target_id,target_symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,TSPAN6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000005,TNMD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000419,DPM1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000457,SCYL3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000460,C1orf112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
ot_sm_druggable.shape

(2294, 7)

In [64]:
ref_ot_merged = ref_table.merge(ot_sm_druggable, on = 'target_id', how = 'left')

In [77]:
ref_ot_merged.query("small_mol_druggable == 'Y' & modality != modality").shape

(39, 18)

In [78]:
ref_ot_merged.query("small_mol_druggable == 'N' & modality == modality").shape

(0, 18)

In [67]:
group_count(ref_ot_merged.query("small_mol_druggable == 'N'"), ['bio_druggable', 'adme_gene'])

,bio_druggable,adme_gene,count
1,Y,N,2026
0,N,Y,121
2,Y,Y,19


In [38]:
ot_druggable_ids = list()

In [ ]:
group_count(tractability_df.query("column_names == 'SM_druggable_family'")\
                           .drop_duplicates(["column_names", 'value', 'target_symbol']),
            ['value'])